In [59]:
import json
import re
from pathlib import Path
from ruamel.yaml import YAML

project_path = Path(r"C:\Users\Owner\Documents\repos\COMP4905\pose-estimation\MousePose-AndreTelfer-2020-05-21")

yaml = YAML()
with open(project_path / 'config.yaml', 'r') as fp:
    config = yaml.load(fp)

# What Frames are Already Labelled?

In [351]:
labeled_data_folder = project_path / "labeled-data"

r = re.compile("img(\d+).png")

def get_labeled_frames():
    labeled_frames = {}
    for video_folder in os.listdir(labeled_data_folder):
        images = os.listdir(labeled_data_folder / video_folder)
        images = list(filter(lambda x: r.match(x), images))
        frames = list(map(lambda x: int(r.match(x).group(1)), images))
        labeled_frames[video_folder] = frames
    
    return labeled_frames

labeled_frames = get_labeled_frames()
print(json.dumps(labeled_frames, indent=4))

{
    "Rat 2 LPS 1": [
        0,
        1323,
        17072,
        19623,
        24044,
        41117,
        51152
    ],
    "Rat 4 Control 1": [
        0,
        6179,
        16331,
        23834,
        41489,
        42372,
        48110,
        52303
    ]
}


# Random Sample Frames

## Select Videos

### Get Videos from Project

In [44]:
import os
import shutil

# Temporarily change the CWD, this lets us use pathlib to resolve relative paths
initial_cwd = os.getcwd()
os.chdir(project_path)
    
videos = list(config['video_sets'].keys())
videos = list(map(lambda x: Path(os.path.abspath(x)), videos))

# Change CWD back
os.chdir(initial_cwd)

videos

[WindowsPath('C:/Users/Owner/Documents/repos/COMP4905/pose-estimation/MousePose-AndreTelfer-2020-05-21/videos/Rat 2 LPS 1.MP4'),
 WindowsPath('C:/Users/Owner/Documents/repos/COMP4905/pose-estimation/MousePose-AndreTelfer-2020-05-21/videos/Rat 4 Control 1.MP4')]

### Specify Videos Manually

In [46]:
video_folder = Path(r"C:\Users\Owner\Documents\repos\COMP4905\videos")
videos = [
    *map(lambda x: video_folder / x, os.listdir(video_folder))
]

# Sample Frames According to Spring Simulation

In [375]:
import cv2
import os
import numpy as np

n_samples = 10
labeled_frames = get_labeled_frames()
frame_name_format = "img{}.png"

for video in videos:
    cap = cv2.VideoCapture(str(video))
    frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    
    name, ext = os.path.splitext(video.name)
    fixed_points = labeled_frames[name]
    
    new_frames = np.random.choice(np.arange(frames+1), size=n_samples).astype(np.float32)
    new_frames = sorted(new_frames)
    
    idx = 0
    print(f"\nNew frames: {new_frames}")
    
    for frame_idx in new_frames:
        cap.set(cv2.CAP_PROP_POS_FRAMES, frame_idx)
        ret, frame = cap.read()
        if not ret:
            print(f"Error reading frame {frame_idx}, skipping...")
            continue
        padded_idx = str(int(frame_idx)).zfill(math.ceil(np.log(frames)/np.log(10))) # Get the frame index with leading 0s (with enough digits for any frame in video)
        file_name = project_path / 'labeled-data' / name / frame_name_format.format(padded_idx)
        file_name = str(file_name)
        print(f"Saving frame {frame_idx} to {file_name}")
        cv2.imwrite(file_name, frame)
        new_frames = new_frames[1:]
    

New frames: [6026.0, 19331.0, 19560.0, 22567.0, 26274.0, 26552.0, 44582.0, 47866.0, 52295.0, 63124.0]
Saving frame 6026.0 to C:\Users\Owner\Documents\repos\COMP4905\pose-estimation\MousePose-AndreTelfer-2020-05-21\labeled-data\Rat 2 LPS 1\img06026.png
Saving frame 19331.0 to C:\Users\Owner\Documents\repos\COMP4905\pose-estimation\MousePose-AndreTelfer-2020-05-21\labeled-data\Rat 2 LPS 1\img19331.png
Saving frame 19560.0 to C:\Users\Owner\Documents\repos\COMP4905\pose-estimation\MousePose-AndreTelfer-2020-05-21\labeled-data\Rat 2 LPS 1\img19560.png
Saving frame 22567.0 to C:\Users\Owner\Documents\repos\COMP4905\pose-estimation\MousePose-AndreTelfer-2020-05-21\labeled-data\Rat 2 LPS 1\img22567.png
Saving frame 26274.0 to C:\Users\Owner\Documents\repos\COMP4905\pose-estimation\MousePose-AndreTelfer-2020-05-21\labeled-data\Rat 2 LPS 1\img26274.png
Saving frame 26552.0 to C:\Users\Owner\Documents\repos\COMP4905\pose-estimation\MousePose-AndreTelfer-2020-05-21\labeled-data\Rat 2 LPS 1\img265

# Test Labeling

In [378]:
import deeplabcut

deeplabcut.label_frames(project_path / "config.yaml")